In [3]:
import numpy as np
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import pandas_ta as ta

def load_data(url):
    """Load dataset from a URL."""
    data = pd.read_csv(url)
    return data

def preprocess_data(data):
    """Preprocess data by selecting features, adding indicators, and normalizing."""
    selected_features = ['close', 'high', 'low', 'open', 'volume', 'MACD', 'RSI', 'ATR', 'BB_upper', 'BB_middle', 'BB_lower']
    data['SMA_10'] = data['close'].rolling(window=10).mean()
    data['SMA_21'] = data['close'].rolling(window=21).mean()
    data['EMA_5'] = ta.ema(data['close'], length=5)
    data['EMA_8'] = ta.ema(data['close'], length=8)
    data['EMA_13'] = ta.ema(data['close'], length=13)
    data = data[selected_features].dropna()
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

def create_sequences(data, sequence_length=45):
    """Create sequences for model training."""
    X, y = [], []
    for i in range(len(data) - sequence_length - 1):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

def split_data(X, y, train_ratio=0.7, val_ratio=0.15):
    """Split data into training, validation, and test sets."""
    train_size = int(len(X) * train_ratio)
    val_size = int(len(X) * val_ratio)
    return (X[:train_size], y[:train_size],
            X[train_size:train_size+val_size], y[train_size:train_size+val_size],
            X[train_size+val_size:], y[train_size+val_size:])

def build_model(input_shape):
    """Build a Bidirectional GRU model."""
    model = Sequential([
        Bidirectional(GRU(256, return_sequences=True, input_shape=input_shape)),
        Dropout(0.18),
        Bidirectional(GRU(512, return_sequences=False)),
        Dropout(0.18),
        Dense(1)
    ])
    optimizer = AdamW(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss=Huber(delta=1.0))
    return model

def train_model(model, X_train, y_train, X_val, y_val, epochs=70, batch_size=24):
    """Train the model with early stopping and learning rate reduction."""
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
    ]
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val), callbacks=callbacks)
    return history

def load_trained_model(model_path):
    """Load a trained model from file."""
    return load_model(model_path)


def evaluate_model(model, X_test, y_test, scaler, X_test_shape):
    """Evaluate the model and calculate performance metrics."""
    y_pred = model.predict(X_test)
    y_pred_actual = scaler.inverse_transform(np.concatenate((y_pred, np.zeros((y_pred.shape[0], X_test_shape[2] - 1))), axis=1))[:, 0]
    y_test_actual = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], X_test_shape[2] - 1))), axis=1))[:, 0]
    
    metrics = {
        'MAE': mean_absolute_error(y_test_actual, y_pred_actual),
        'RMSE': np.sqrt(mean_squared_error(y_test_actual, y_pred_actual)),
        'MAPE': mean_absolute_percentage_error(y_test_actual, y_pred_actual) * 100,
        'R²': r2_score(y_test_actual, y_pred_actual),
        'RMSE %': (np.sqrt(mean_squared_error(y_test_actual, y_pred_actual)) / np.mean(y_test_actual)) * 100
    }
    return metrics, y_test_actual, y_pred_actual

def plot_predictions(y_test_actual, y_pred_actual):
    """Plot actual vs predicted stock prices."""
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_actual, label='Actual')
    plt.plot(y_pred_actual, label='Predicted')
    plt.title('Next-Day Stock Price Prediction (GRU)')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
def flow():
    # Execution
    url = "https://raw.githubusercontent.com/ranjithkumar5807/stock-prediction/refs/heads/main/data/test/technical_indicators/TCS.NS_indicators.csv"
    model_path="https://github.com/ranjithkumar5807/stock-prediction/blob/ddbf3c73929026e3405acdcbb0dbe8876d723bb4/Model/TCS_mae%2047rmse63%20r2%2097%20mape1.36.keras"
    data = load_data(url)
    scaled_data, scaler = preprocess_data(data)
    X, y = create_sequences(scaled_data)
    # X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y)
    # model = build_model((X_train.shape[1], X_train.shape[2]))
    # history = train_model(model, X_train, y_train, X_val, y_val)
    model=load_trained_model(model_path)
    # metrics, y_test_actual, y_pred_actual = evaluate_model(model, X_test, y_test, scaler, X_test.shape)
    metrics, y_test_actual, y_pred_actual = evaluate_model(model, X, y, scaler, X.shape)
    
    plot_predictions(y_test_actual, y_pred_actual)
    
    # Print Evaluation Metrics
    for key, value in metrics.items():
        print(f'{key}: {value}')


In [ ]:
flow()

In [ ]:
pip install tensorflow